## Link Prediction using Graph Neural Networks 

> `Link Prediction`은 node와 node 간의 연결성이 존재하는지 예측하는 것입니다. 예를 들어, 추천 시스템에서 사용자가 제품을 구매할 것인지 혹은 사용자가 제품을 구매할 때 평점을 얼마나 남길지를 예측하는 것을 의미합니다. 가장 대표적인 모델은 `SEAL`이 있습니다. 

In [1]:
import dgl 
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import itertools 
import numpy as np 
import scipy.sparse as sp 

c:\Users\EonKim\anaconda3\envs\dgl\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import dgl.data 

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [3]:
u, v = g.edges()

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids) # edge 중 random으로 추출.
test_size = int(len(eids) * 0.1) # 10%를 testset으로 사용.
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]] # train, test split

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

In [4]:
train_g = dgl.remove_edges(g, eids[:test_size])

In [5]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [6]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

In [7]:
# GraphSAGE 모델은 마지막 Layer에서 not product, MLP, LSTM 등 다양한 모델을 사용합니다. 
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [8]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

## Train loop 
### $$ \mathcal{L} = - \sum_{u \sim v \in \mathcal{D}} (y_{u \sim v} \log ( \hat{y}_{u \sim v}) + (1-\hat{y}_{u \sim v}) \log (1-\hat{y}_{u \sim v})) $$

> `Link Prediction`은 연결이 되었는지 혹은 연결되지 않았는지를 예측하기 때문에, 이진분류에서 사용하는 `Binary Cross Entropy`를 사용합니다.

In [10]:
from sklearn.metrics import roc_auc_score

model = GraphSAGE(train_g.ndata['feat'].shape[1], 16)
# You can replace DotPredictor with MLPPredictor.
#pred = MLPPredictor(16)
pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [11]:
# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(100):
    # forward
    h = model(train_g, train_g.ndata['feat'])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))


# Thumbnail credits: Link Prediction with Neo4j, Mark Needham
# sphinx_gallery_thumbnail_path = '_static/blitz_4_link_predict.png'

c:\Users\EonKim\anaconda3\envs\dgl\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 0, loss: 0.6930160522460938
In epoch 5, loss: 0.6591872572898865
In epoch 10, loss: 0.5690920352935791
In epoch 15, loss: 0.5269290804862976
In epoch 20, loss: 0.4924691319465637
In epoch 25, loss: 0.46520406007766724
In epoch 30, loss: 0.44382765889167786
In epoch 35, loss: 0.42137569189071655
In epoch 40, loss: 0.39889687299728394
In epoch 45, loss: 0.3788817524909973
In epoch 50, loss: 0.35822540521621704
In epoch 55, loss: 0.33676764369010925
In epoch 60, loss: 0.3156810998916626
In epoch 65, loss: 0.294638991355896
In epoch 70, loss: 0.2731066942214966
In epoch 75, loss: 0.2520308792591095
In epoch 80, loss: 0.230819433927536
In epoch 85, loss: 0.21007879078388214
In epoch 90, loss: 0.18985556066036224
In epoch 95, loss: 0.1700807362794876
AUC 0.8541407425709217
